<a href="https://colab.research.google.com/github/visheshrao17/CIFAR-Experimentation-/blob/main/CNN_CIAFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset


transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

print("Libraries imported and transformations defined.")

Libraries imported and transformations defined.


In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train_subset_size = 10000
test_subset_size = 2000

# Create a subset of the training dataset
train_indices = torch.randperm(len(trainset))[:train_subset_size]
train_subset = Subset(trainset, train_indices)

# Create a subset of the test dataset
test_indices = torch.randperm(len(testset))[:test_subset_size]
test_subset = Subset(testset, test_indices)

# Define batch size
batch_size = 64

# Create DataLoader instances
trainloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

print(f"CIFAR-10 training dataset loaded with {len(train_subset)} samples.")
print(f"CIFAR-10 test dataset loaded with {len(test_subset)} samples.")
print(f"DataLoader instances created with batch size {batch_size}.")

100%|██████████| 170M/170M [00:01<00:00, 88.5MB/s] 


CIFAR-10 training dataset loaded with 10000 samples.
CIFAR-10 test dataset loaded with 2000 samples.
DataLoader instances created with batch size 64.


In [5]:
import torch.nn as nn
import torch.nn.functional as F

# Define a Convolutional Neural Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.pool = nn.MaxPool2d(2, 2) 
        self.conv2 = nn.Conv2d(6, 16, 5) 
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

print("CNN model architecture defined.")

CNN model architecture defined.


In [6]:
import torch.optim as optim

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()  
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  

        outputs = model(inputs) 
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step() 

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples
    return epoch_loss, epoch_accuracy

def validate_epoch(model, dataloader, criterion, device):
    model.eval()  
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples
    return epoch_loss, epoch_accuracy

print("Training and validation functions defined.")

Training and validation functions defined.


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define hyperparameters to experiment with
learning_rates = [0.01, 0.001]
batch_sizes = [32, 64]
num_epochs_list = [5, 10]

best_val_accuracy = 0.0
best_hyperparameters = {}

results = []

for lr in learning_rates:
    for bs in batch_sizes:
        for epochs in num_epochs_list:
            print(f"\n--- Experimenting with LR: {lr}, Batch Size: {bs}, Epochs: {epochs} ---")

            # Re-initialize model and optimizer for each experiment
            model = Net().to(device)
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

            # Create new DataLoader instances with current batch size
            trainloader_exp = DataLoader(train_subset, batch_size=bs, shuffle=True)
            testloader_exp = DataLoader(test_subset, batch_size=bs, shuffle=False)

            # Lists to store metrics for the current hyperparameter combination
            current_train_losses = []
            current_train_accuracies = []
            current_val_losses = []
            current_val_accuracies = []
            
            for epoch in range(epochs):
                # Train the model for one epoch
                train_loss, train_acc = train_epoch(model, trainloader_exp, criterion, optimizer, device)
                current_train_losses.append(train_loss)
                current_train_accuracies.append(train_acc)

                # Validate the model
                val_loss, val_acc = validate_epoch(model, testloader_exp, criterion, device)
                current_val_losses.append(val_loss)
                current_val_accuracies.append(val_acc)

                print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            # Store results for this combination
            results.append({
                'learning_rate': lr,
                'batch_size': bs,
                'num_epochs': epochs,
                'train_losses': current_train_losses,
                'train_accuracies': current_train_accuracies,
                'val_losses': current_val_losses,
                'val_accuracies': current_val_accuracies
            })

            # Check if this model is the best performing one
            if current_val_accuracies[-1] > best_val_accuracy:
                best_val_accuracy = current_val_accuracies[-1]
                best_hyperparameters = {
                    'learning_rate': lr,
                    'batch_size': bs,
                    'num_epochs': epochs
                }
                # Optionally, save the best model state here
                # torch.save(model.state_dict(), 'best_model.pth')

print(f"\nBest Validation Accuracy: {best_val_accuracy:.4f} with Hyperparameters: {best_hyperparameters}")

Using device: cpu

--- Experimenting with LR: 0.01, Batch Size: 32, Epochs: 5 ---
Epoch 1/5 | Train Loss: 2.1406, Train Acc: 0.1984 | Val Loss: 1.9189, Val Acc: 0.2840
Epoch 2/5 | Train Loss: 1.7814, Train Acc: 0.3357 | Val Loss: 1.6330, Val Acc: 0.3900
Epoch 3/5 | Train Loss: 1.5805, Train Acc: 0.4202 | Val Loss: 1.5990, Val Acc: 0.4225
Epoch 4/5 | Train Loss: 1.4593, Train Acc: 0.4636 | Val Loss: 1.4839, Val Acc: 0.4685
Epoch 5/5 | Train Loss: 1.3847, Train Acc: 0.4968 | Val Loss: 1.4626, Val Acc: 0.4750

--- Experimenting with LR: 0.01, Batch Size: 32, Epochs: 10 ---
Epoch 1/10 | Train Loss: 2.1989, Train Acc: 0.1627 | Val Loss: 1.9710, Val Acc: 0.2690
Epoch 2/10 | Train Loss: 1.8245, Train Acc: 0.3199 | Val Loss: 1.6676, Val Acc: 0.3655
Epoch 3/10 | Train Loss: 1.5881, Train Acc: 0.4179 | Val Loss: 1.5921, Val Acc: 0.4145
Epoch 4/10 | Train Loss: 1.4716, Train Acc: 0.4590 | Val Loss: 1.4476, Val Acc: 0.4675
Epoch 5/10 | Train Loss: 1.3840, Train Acc: 0.4944 | Val Loss: 1.5845, Val 

## Baseline Model Results Summary

In [10]:
import pandas as pd

# Create summary of baseline experiments (without augmentation)
summary_data = []

for res in results:
    summary_data.append({
        'Learning Rate': res['learning_rate'],
        'Batch Size': res['batch_size'],
        'Num Epochs': res['num_epochs'],
        'Final Val Accuracy': res['val_accuracies'][-1],
        'Final Val Loss': res['val_losses'][-1]
    })

summary_df = pd.DataFrame(summary_data)

print("\n=== Baseline Model Results (Without Data Augmentation) ===")
display(summary_df.round(4))

print(f"\nBest Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Best Hyperparameters: {best_hyperparameters}")


=== Baseline Model Results (Without Data Augmentation) ===


,Learning Rate,Batch Size,Num Epochs,Final Val Accuracy,Final Val Loss
0,0.010,32,5,0.4750,1.4626
1,0.010,32,10,0.5175,1.4401
2,0.010,64,5,0.4490,1.5031
3,0.010,64,10,0.5335,1.3538
4,0.001,32,5,0.2370,2.0283
5,0.001,32,10,0.4045,1.6298
6,0.001,64,5,0.1280,2.2971
7,0.001,64,10,0.1890,2.1929



Best Validation Accuracy: 0.5335
Best Hyperparameters: {'learning_rate': 0.01, 'batch_size': 64, 'num_epochs': 10}
